<a href="https://colab.research.google.com/github/EdwardKG/Angular-Slider/blob/main/Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import numpy as np


# define a CNN architecture
class FaceNet(nn.Module):
    def __init__(self):
        super().__init__()
        # block 1
        self.conv_1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)

        self.bn_1 = nn.BatchNorm2d(16)
        self.conv_12 = nn.Conv2d(in_channels=16, out_channels=16,
                                kernel_size=1, stride=1, padding=0)
        self.bn_12 = nn.BatchNorm2d(16)

        # block 2
        self.conv_2 = nn.Conv2d(in_channels=16, out_channels=32,
                                kernel_size=3, stride=1, padding=1)
        self.bn_2 = nn.BatchNorm2d(32)
        self.conv_22 = nn.Conv2d(in_channels=32, out_channels=32,
                                kernel_size=1, stride=1, padding=0)
        self.bn_22 = nn.BatchNorm2d(32)

        # block 3
        self.conv_3 = nn.Conv2d(in_channels=32, out_channels=64,
                                kernel_size=3, stride=1, padding=1)
        self.bn_3 = nn.BatchNorm2d(64)
        self.conv_32 = nn.Conv2d(in_channels=64, out_channels=64,
                                kernel_size=1, stride=1, padding=0)
        self.bn_32 = nn.BatchNorm2d(64)

        # block 4
        self.conv_4 = nn.Conv2d(in_channels=64, out_channels=128,
                                kernel_size=3, stride=1, padding=1)
        self.bn_4 = nn.BatchNorm2d(128)
        self.conv_42 = nn.Conv2d(in_channels=128, out_channels=128,
                                kernel_size=1, stride=1, padding=0)
        self.bn_42 = nn.BatchNorm2d(128)

        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(p=0.2)

        self.linear_1 = nn.Linear(2048, 128)
        self.bn_linear = nn.BatchNorm1d(128)

        self.linear_2 = nn.Linear(128, 10)

        self.relu = nn.ReLU()

        # image size: 3, 32, 32, 4 layers pooling 2, stripe 2. 32(2^4)
        # add cosine similarity layer
        self.cos_sim = nn.CosineSimilarity(dim=1, eps=1e-6)

    def reshape_input(self, x):
        # reshape input tensor to (batch_size, channels, height, width)
        x = x.permute(0, 3, 1, 2)
        return x

    def forward(self, x):
        x = self.reshape_input(x)
        x = self.conv_1(x)
        x = self.bn_1(x)
        x = self.relu(x)
        x = self.conv_12(x)
        x = self.bn_12(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.dropout(x)


        x = self.conv_2(x)
        x = self.bn_2(x)
        x = self.relu(x)
        x = self.conv_22(x)
        x = self.bn_22(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.dropout(x)

        x = self.conv_3(x)
        x = self.bn_3(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.dropout(x)
        x = self.conv_32(x)
        x = self.bn_32(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.dropout(x)

        x = self.conv_4(x)
        x = self.bn_4(x)
        x = self.relu(x)
        x = self.conv_42(x)
        x = self.bn_42(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.dropout(x)

        x = torch.flatten(x, start_dim=1, end_dim=-1)

        x = self.linear_1(x)
        x = self.bn_linear(x)
        e = self.relu(x)
        x = self.dropout(e)
        y = self.linear_2(x)


        return y, e


# load and preprocess the dataset
lfw_dataset = datasets.ImageFolder("/content/drive/MyDrive/lfw2/lfw2", transform=transforms.Compose([
    transforms.Resize(128),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
]))

# split the dataset into training and testing sets
train_set, test_set = torch.utils.data.random_split(lfw_dataset, [len(lfw_dataset)-500, 500])

# define a dataloader to load images in batches
train_loader = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True, num_workers=4, pin_memory=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=64, shuffle=False, num_workers=4, pin_memory=True)


facenet = FaceNet().to('cuda')

# define the loss function and optimizer
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(facenet.parameters(), lr=0.01, momentum=0.9)
criterion = nn.TripletMarginLoss().to('cuda')
optimizer = optim.Adam(facenet.parameters(), lr=0.001)



# define the training loop
def train_loop(model, train_loader, criterion, optimizer):
    model.train()
    running_loss = 0.0
    for batch_idx, data in enumerate(train_loader):
        print(inputs.shape) # try and recalculate
        inputs, targets = data
        inputs = inputs.reshape(inputs.shape[0], 1, 128, 128)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    train_loss = running_loss / len(train_loader)
    return train_loss

# define the testing loop
def test_loop(model, test_loader, criterion):
  model.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for batch_idx, (anchor_imgs, pos_imgs, neg_imgs) in enumerate(test_loader):
      anchor_embs = model(anchor_imgs)
      pos_embs = model(pos_imgs)
      neg_embs = model(neg_imgs)
      loss = criterion(anchor_embs, pos_embs, neg_embs)
      test_loss += loss.item()
      test_loss /= len(test_loader.dataset)
      print(f'Test set: Average loss: {test_loss:.4f}')

#train the model
for epoch in range(10):
   # train the model
    train_loss = train_loop(facenet, train_loader, criterion, optimizer)

    # test the model
    test_loss = test_loop(facenet, test_loader, criterion)

    # print the training and testing loss and accuracy for each epoch
    print(f"Epoch {epoch+1}/{10}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Test Acc:")

# #save the trained model

# torch.save(facenet.state_dict(), 'facenet.pth')
